# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import nltk
from nltk import word_tokenize
import time
import re
from sklearn.externals import joblib
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
pd.set_option('mode.chained_assignment', None)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_messages.db')
df = pd.read_sql_table(table_name='messages_categories',con=engine)

In [3]:
X = df.message
y = df.iloc[:,3:]
X.shape,y.shape

((26216,), (26216, 35))

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    words = word_tokenize(text)
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42))),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
start = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=0.3)
pipeline.fit(X_train, y_train)
end = time.time()
time_elapsed = (end - start)/60
print('Training data size:{} documents.'.format(X_train.shape[0]))
print('Baseline model took {} minutes to train.'.format(round(time_elapsed,2)))

Training data size:18351 documents.
Baseline model took 1.13 minutes to train.


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_pred,y_test,target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.94      0.81      0.87      6850
               request       0.37      0.83      0.51       586
                 offer       0.00      0.00      0.00         0
           aid_related       0.52      0.75      0.61      2218
          medical_help       0.07      0.58      0.12        76
      medical_products       0.07      0.75      0.13        40
     search_and_rescue       0.05      0.56      0.10        18
              security       0.00      0.00      0.00         2
              military       0.09      0.62      0.15        34
                 water       0.25      0.78      0.37       158
                  food       0.34      0.79      0.48       382
               shelter       0.25      0.79      0.38       224
              clothing       0.06      1.00      0.11         7
                 money       0.05      0.67      0.09        12
        missing_people       0.02      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {
    'clf__estimator__min_samples_leaf' : [1,5,10],
    'clf__estimator__max_features' : ["auto",'log2'] 
}
cv = GridSearchCV(
    estimator = pipeline, 
    param_grid = parameters, 
    cv = 3,
    n_jobs = -1,
    scoring = 'f1_samples',
    return_train_score = True
)
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
tuned_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(
            RandomForestClassifier(
                random_state=42,
                max_features = 'auto',
                min_samples_leaf = 1,
                n_estimators = 200
                )
            )
        ),
    ])
tuned_pipeline.fit(X_train, y_train)
y_pred_tuned = tuned_pipeline.predict(X_test)
print(classification_report(y_pred_tuned,y_test,target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.97      0.80      0.88      7211
               request       0.42      0.89      0.57       626
                 offer       0.00      0.00      0.00         0
           aid_related       0.61      0.78      0.69      2507
          medical_help       0.07      0.72      0.12        60
      medical_products       0.07      0.79      0.14        39
     search_and_rescue       0.03      0.50      0.05        10
              security       0.00      0.00      0.00         1
              military       0.04      0.64      0.07        14
                 water       0.26      0.90      0.40       142
                  food       0.43      0.89      0.58       429
               shelter       0.25      0.82      0.38       215
              clothing       0.06      0.88      0.11         8
                 money       0.05      0.80      0.09        10
        missing_people       0.02      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [7]:
improved_pipeline = Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize,stop_words='english')),
        (
            'clf', MultiOutputClassifier(
                AdaBoostClassifier(
                    random_state = 42,
                    learning_rate = 0.3,
                    n_estimators = 200
                )
            )
        ),
    ])
improved_pipeline.fit(X_train, y_train)
y_pred_improved = improved_pipeline.predict(X_test)
print(classification_report(y_pred_improved,y_test,target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.97      0.79      0.87      7259
               request       0.46      0.83      0.59       738
                 offer       0.00      0.00      0.00         5
           aid_related       0.58      0.78      0.67      2408
          medical_help       0.17      0.61      0.27       178
      medical_products       0.22      0.73      0.34       128
     search_and_rescue       0.12      0.60      0.21        40
              security       0.00      0.00      0.00         3
              military       0.17      0.53      0.26        80
                 water       0.64      0.73      0.68       437
                  food       0.71      0.83      0.76       756
               shelter       0.50      0.83      0.62       426
              clothing       0.38      0.68      0.49        65
                 money       0.23      0.57      0.33        68
        missing_people       0.12      

### 9. Export your model as a pickle file

In [8]:
joblib.dump(improved_pipeline, 'model.pkl')

['model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.